# Practice Session 09: Data streams

In this session we will take a large corpus of tweets (just the author and the hashtags) and compute some statistics using data streams methods.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Author: <font color="blue">Your name here</font>

E-mail: <font color="blue">Your e-mail here</font>

Date: <font color="blue">The current date here</font>

In [ ]:
import io
import gzip
import csv
import math
import random
import statistics
import secrets

# 0. Dataset and how to iterate

The input file contain about 35,500 messages ("tweets") posted between March 13th, 2020, and March 14th, 2020, containing a hashtag or keyword related to COVID-19, and posted by a user declaring a location in Catalonia. This is a simplified file in which only the author of the tweet (`@user`) and a hashtag (`#hashtag`) are icluded.

During this practice, we will **never** load this file in main memory.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
INPUT_FILE = "CovidLockdownCatalonia-hashtags.csv.gz"

with gzip.open(INPUT_FILE, "rt") as file:
    reader = csv.reader(file, delimiter="\t")
    for (username, hashtag) in reader:
        # Prints 0.05% of lines
        if random.random() < 0.0005:
            print("User %s used hashtag %s" % (username, hashtag)) 

# 1. Determine approximately the top-5 hashtags

In this dataset the most frequent hashtags are:

* #Covid_19
* #COVIDー19
* #Coronavirus
* #COVID19
* #coronavirus

Instead of loading the entire dataset in main memory, we will use reservoir sampling to determine approximately the top-5 hashtags.

**Reservoir sampling**: In reservoir sampling, if we have a reservoir of size S:

* We store the first S elements of the stream
* When the n<sup>th</sup> element arrives (let's call it X<sub>n</sub>):
   * With probability 1 - s/n, we ignore this element.
   * With probability s/n, we:
      * Discard a random element from the reservoir
      * Add element X<sub>n</sub> to the reservoir (calling *add_to_reservoir*)
      
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Implement a function `add_reservoir(reservoir, item, max_size)` that adds an item to the reservoir, maintaining its size. If the reservoir is already of size *max_size*, a random item is selected and evicted *before* adding the item. It is important to evict an old item *before* adding the new item. Use the following skeleton:

```python

def add_to_reservoir(reservoir, item, max_reservoir_size):
    # YOUR CODE HERE
    assert(len(reservoir) <= max_reservoir_size)
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

Create a function to iterate through the file using the reservoir sampling method seen in class. In this function you will decide, for every item, whether to call *add_to_reservoir* or to ignore the item.

You can use the following skeleton:

```python
def reservoir_sampling(filename, reservoir_size):
    reservoir = []

    with gzip.open(filename, "rt") as file:
        reader = csv.reader(file, delimiter="\t")
        i = 0
        for username, hashtag in reader:
            i += 1
            
            # YOUR CODE HERE

    return (i, reservoir)
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

Test your function using the following code:

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
reservoir_size = 30
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

The reservoir contains repeated items. You can compute the absolute frequencies of the top 5 using the following code.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, hashtag) for hashtag, frequency in freq.items()], reverse=True)[:5]
for absolute_frequency, hashtag in most_frequent_items:
    print("%d %s" % (absolute_frequency, hashtag))

Write code to compute the 5 most frequent hashtags in the reservoir and their relative frequencies, as percentages.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to print the top hashtags and their relative frequencies</font>

If you see an item C times in the reservoir, you can estimate the item appears *C x dataset_size / reservoir_size* times in the entire dataset (*dataset_size* is the size of the entire dataset). 

For various sizes of the reservoir: 10, 20, 50, 100, 200, 500, 1000, list the top-5 hashtags and your estimate of their frequency in the entire dataset.
 
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to try different reservoir sizes</font>

Find by trial and error, and include in your report, the minimum reservoir size you would have to use to have an overlap of 3/5 between the queries found by the approximate method and the actual top-5.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with a brief commentary indicating what reservoir size you should be using if you want a 3/5 overlap among the hashtags you find and the actual top-5.</font>

# 2. Determine approximately the number of users


We will estimate the number of distinct users without creating a dictionary or hash table with users, but instead, we will use the Flajolet-Martin probabilistic counting method.

**Flajolet-Martin probabilistic counting**:

* For several passes
   * Create hash funcion h
   * For every element *u* in the stream:
      * Compute hash value *h(u)*
      * Let *r(u)* be the number of trailing zeroes in *h(u)*
      * Maintain *R* as the maximum value of *r(u)* seen so far
   * Add *2<sup>R</sup>* as an estimate for the number of distinct elements *u* seen
* The final estimate is the average or the median of the estimates found in each pass

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Use this function to count trailing zeroes in the binary representation of a number.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

Use this function to generate a random hash function. Note this generates a function, so you can do `hash_function = random_hash_function()` and then call `hash_function(x)` to compute the hash value of `x`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

Perform *number_of_passes* passes over the file, reading the entire file on each pass (we don't use the reservoir in this part). In each pass, create a new hash function and use it to hash userids. Keep the maximum number of trailing zeroes seen in the hash value of a userid. 

```python
number_of_passes = 10
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    
    estimates.append(estimate)
    print("Estimate on pass %d: %d" % (i+1, estimate))
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [ ]:
print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

Compute the median of estimates obtained in 3 separate runs of your algorithm; each run should do 10 passes over the file. 

Increase the numbe of passes to 20 and perform 3 separate runs.

*Note: in this dataset, the actual number of different users is approximately 7,200.*

<font size="+1" color="red">Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

# DELIVER (individually)

Remember to read the section on "delivering your code" in the [course evaluation guidelines](https://github.com/chatox/data-mining-course/blob/master/upf/upf-evaluation.md).

Deliver a zip file containing:

* This notebook

## Extra points available

For more learning and extra points, process the `DCEP/strip/xml/EN/REPORT` subdirectory of the *Digital Corpus of the European Parliament*, which is inside [DCEP-strip-EN-pub.tar.bz2](https://wt-public.emm4u.eu/Resources/DCEP-2013/strip/DCEP-strip-EN-pub.tar.bz2) in this [download page](https://wt-public.emm4u.eu/Resources/DCEP-2013/DCEP-Download-Page.html). You will need `bunzip2` to decompress and `tar` to extract, and then you will need to either iterate through the corresponding directory or concatenate everything in a file. 

What we want to find out is the top 20 words used in these reports, obtained using reservoir sampling. To read word by word you can use `for line in input_file` and `for word in line.split()`.

**Note:** if you go for the extra points, add ``<font size="+2" color="blue">Additional results: EU Parliament Documents</font>`` at the top of your notebook. **Do not include your input file with your submission**, instead, leave it in some file sharing site such as TransferNow or similar, from where the instructor can download it.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>